Split AVL Route points into N and S points<br>

Split GTFS routes into N and S routes<br>
Create Routes layer for the routes*<br>

Create a route from line features

In [ ]:
import arcpy
arcpy.CreateFileGDB_management(r"C:\Users\Cole\Desktop\Spring2022\AVLResearch\Data\Linear_Ref_Files", "LRF.gdb")

In [ ]:
import pandas as pd
AVLs = r'C:\Users\Cole\Desktop\Spring2022\AVLResearch\Data\Aline-vehicle-messages-Oct1_Oct8_2016\Aline-vehicle-messages-Oct1_Oct8-2016.csv'
GTFS = r':\Users\Cole\Desktop\Spring2022\AVLResearch\Data\GTFS_Query'

AVL_Points = pd.read_csv(AVLs, index_col = False)

In [ ]:
#sort avl rows by host, UTC time, local time
AVL_Points_Sort = AVL_Points.sort_values(by =['SOURCE_HOST','MESSAGE_TIMESTAMP','LOCAL_TIMESTAMP'])

In [ ]:
new = AVL_Points_Sort[["SOURCE_HOST", 'LONGITUDE',"LATITUDE",
                       'LOCAL_TIMESTAMP', 'MESSAGE_TIMESTAMP',
                       'MESSAGE_TYPE_TEXT', 'DIRECTION',
                      'ODOMETER', 'FOM']]
#this takes a section of 1 north and 1 south trip from AVL
AVL_Points_Select = new[101:209]

In [ ]:
#translate Lat and Long to actual coordinates
AVL_Points_Select["LongCoord"] = AVL_Points_Select['LONGITUDE']/10000000
AVL_Points_Select["LatCoord"] = AVL_Points_Select['LATITUDE']/10000000

In [ ]:
#push selection to csv
AVL_Points_Select.to_csv(r"C:\Users\Cole\Desktop\Spring2022\AVLResearch\Data\Linear_Ref_Files\AVL_Pts.csv")

In [ ]:
arcpy.env.workspace = r"C:\Users\Cole\Desktop\Spring2022\AVLResearch\Data\Linear_Ref_Files"
#turn avl points tavle into points feature class
arcpy.management.XYTableToPoint(r"AVL_Pts.csv", "LRF.gdb\AVL_Pts_shp",
                                "LongCoord", "LatCoord", "")
    
    
input_features = "LRF.gdb\AVL_Pts_shp"

# output data
output_feature_class = r"LRF.gdb\AVL_Pts_Proj"

# create a spatial reference object for the output coordinate system
out_coordinate_system = arcpy.SpatialReference('NAD 1983 NSRS2007 StatePlane Minnesota Central FIPS 2202 (Meters)')

# run the tool
arcpy.Project_management(input_features, output_feature_class, out_coordinate_system)
#we now have a projected version of the AVLpoints class

In [ ]:
#GTFS shapes to features
#reproject
#turn the gtfs N trip shape and S trip shape into polylines
arcpy.conversion.GTFSShapesToFeatures(r"C:\Users\Cole\Desktop\Spring2022\AVLResearch\Data\GTFS_Query\shapes.txt",
                                      r"C:\Users\Cole\Desktop\Spring2022\AVLResearch\Data\Linear_Ref_Files\LRF.gdb\GTFSShapes")

In [ ]:
input_features = "LRF.gdb\GTFSShapes"

# output data
output_feature_class = r"LRF.gdb\GTFS_Shapes_Proj"

# create a spatial reference object for the output coordinate system
out_coordinate_system = arcpy.SpatialReference('NAD 1983 NSRS2007 StatePlane Minnesota Central FIPS 2202 (Meters)')

# run the tool
arcpy.Project_management(input_features, output_feature_class, out_coordinate_system)
#now have GTFS points in projected form

In [ ]:
# Set the workspace
arcpy.env.workspace = r'C:\Users\Cole\Desktop\Spring2022\AVLResearch\Data\Linear_Ref_Files\LRF.gdb'

query = "\"shape_id\" = \'9210001\'"
# Within selected features, further select only only N trip points (shape 921001) 
select = arcpy.SelectLayerByAttribute_management('GTFS_Shapes_Proj', 'NEW_SELECTION', 
                                        query)

# Write the selected features to a new feature class
arcpy.CopyFeatures_management(select, 'GTFS_Shapes_Proj_N')

query = "\"shape_id\" = \'9210002\'"
# Within selected features, further select only only S trip points (shape 921002) 

select = arcpy.SelectLayerByAttribute_management('GTFS_Shapes_Proj', 'NEW_SELECTION', 
                                        query)

# Write the selected features to a new feature class
arcpy.CopyFeatures_management(select, 'GTFS_Shapes_Proj_S')

In [ ]:
#same as previous cell except splitting AVL data instead
# Set the workspace
arcpy.env.workspace = r'C:\Users\Cole\Desktop\Spring2022\AVLResearch\Data\Linear_Ref_Files\LRF.gdb'
query = "\"DIRECTION\" = 4"   
select = arcpy.SelectLayerByAttribute_management('AVL_Pts_Proj', 'NEW_SELECTION', 
                                        query)

# Write the selected features to a new feature class
arcpy.CopyFeatures_management(select, 'AVL_Pts_Proj_N')

query = "\"DIRECTION\" = 1"

select = arcpy.SelectLayerByAttribute_management('AVL_Pts_Proj', 'NEW_SELECTION', query)

# Write the selected features to a new feature class
arcpy.CopyFeatures_management(select, 'AVL_Pts_Proj_S')

In [ ]:
#create route layer from N trip GTFS
in_lines = r"C:\Users\Cole\Desktop\Spring2022\AVLResearch\Data\Linear_Ref_Files\LRF.gdb\GTFS_Shapes_Proj_N"
rid = 'shape_id'
out_routes = r"C:\Users\Cole\Desktop\Spring2022\AVLResearch\Data\Linear_Ref_Files\LRF.gdb\RouteN"


arcpy.CreateRoutes_lr(in_lines, rid, out_routes, "LENGTH", "#", "#", 
                      "LOWER_LEFT", 0.00018939394)

In [ ]:
#create route layer from S trip GTFS
in_lines = r"C:\Users\Cole\Desktop\Spring2022\AVLResearch\Data\Linear_Ref_Files\LRF.gdb\GTFS_Shapes_Proj_S"
rid = 'shape_id'
out_routes = r"C:\Users\Cole\Desktop\Spring2022\AVLResearch\Data\Linear_Ref_Files\LRF.gdb\RouteS"


arcpy.CreateRoutes_lr(in_lines, rid, out_routes, "LENGTH", "#", "#", 
                      "UPPER_RIGHT", 0.00018939394)

In [ ]:
#the next 2 cells can be switched out for N or S, just replace
import arcpy
#locate features along route
arcpy.env.workspace = r'C:\Users\Cole\Desktop\Spring2022\AVLResearch\Data\Linear_Ref_Files\LRF.gdb'


# Set local variables
feats = "AVL_Pts_Proj_N"
rts = "RouteN"
rid = "shape_id"
tol = "9 Meters"
tbl = "locate_events_N"
props = "shape_id POINT mp" 

# Execute LocateFeaturesAlongRoutes for 
arcpy.LocateFeaturesAlongRoutes_lr(feats,
                                   rts,
                                   rid,
                                   tol,
                                   tbl,
                                   props,
                                   route_locations = True,
                                   distance_field = True,
                                   m_direction_offsetting = True)

In [1]:
import arcpy
arcpy.env.workspace = r'C:\Users\Cole\Desktop\Spring2022\AVLResearch\Data\Linear_Ref_Files\LRF.gdb'

# Set local variables
rt = "RouteN"  
rid = "shape_id" 
tbl = "locate_events_N"
props = "shape_id POINT mp" 
lyr = r'C:\Users\Cole\Desktop\Spring2022\AVLResearch\Data\Linear_Ref_Files\out.shp'

# Execute MakeRouteEventLayer
x = arcpy.MakeRouteEventLayer_lr(rt, rid, tbl, props, lyr, "#", "ERROR_FIELD")
x

<Result 'C:\\Users\\Cole\\Desktop\\Spring2022\\AVLResearch\\Data\\Linear_Ref_Files\\out.shp'>